# nginx-作为负载均衡器

nginx是以高并发和内存占用少出名，它是一个http服务器，也是反向代理服务器，它更是负载均衡器。

作为负载均衡器，在版本1.9之前，它只能作为http的负载均衡，也就是在网络模型的第七层发挥作用，1.9之后，它可以对tcp进行负载均衡，比如redis，mysql等。

nginx的负载均衡是出了名的简单，它跟反向代理的功能是紧密结合在一起的。比如下面是一个网站上的一段配置：

```
upstream rails365 {
    # Path to Unicorn SOCK file, as defined previously
    server unix:///home/xxx/rails365/shared/tmp/sockets/unicorn.sock fail_timeout=0;
}

server {
    listen 80 default_server;
    # listen [::]:80 default_server ipv6only=on;
    server_name www.rails365.net;
    root         /home/xxx/rails365/current/public;
    keepalive_timeout 70;

    location ~ ^/assets/ {
       gzip_static on;
       expires max;
       add_header Cache-Control public;

       # add_header ETag "";
       # break;
    }

        try_files $uri/index.html $uri @rails365;
    location @rails365 {
          proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
          proxy_set_header Host $http_host;
          proxy_redirect off;
          proxy_pass http://rails365;
    }
    # redirect server error pages to the static page /50x.html
    error_page   500 502 503 504  /50x.html;
    location = /50x.html {
        root   html;
    }
}
```

服务器上部署的是`ruby on rails`项目，用`unicorn`来跑ruby的代码，它是监听在一个`unix socket`上的，也就是`unix:///home/xxx/rails365/shared/tmp/sockets/unicorn.sock`这个文件，`proxy_pass http://rails365;`是反向代理到上游服务器，也就是unicorn的部分，upstream这里指的就是上游服务器的部分。

### 设置负载均衡-server指令

要实现负载均衡很简单，部署时多一个unicorn进程，监听另外一个`unix socket`，就等于多了一台服务器，只需这样做：
```
upstream rails365 {
    # Path to Unicorn SOCK file, as defined previously
    server unix:///home/xxx/rails365/shared/tmp/sockets/unicorn.sock;
    server unix:///home/xxx/rails365_cap/shared/tmp/sockets/unicorn.sock;
}
```

很简单，只要用`server指令`添加多一行服务器就可以了。

现在有两个上游服务器了，那么以什么样的方式访问这两个上游服务器的呢?

默认情况下，如果不指定方式，就是随机轮循(round-robin)。

### 配置参数

1. round-robin: 随机轮循
```
upstream rails365 {
    round-robin;
    server unix:///home/yinsigan/rails365/shared/tmp/sockets/unicorn.sock;
    server unix:///home/yinsigan/rails365_cap/shared/tmp/sockets/unicorn.sock;
}
```

2. least_conn: 优先发送给那些接受请求少的，目的是为了让请求分发得更平衡些
```
upstream rails365 {
    least_conn;
    server unix:///home/yinsigan/rails365/shared/tmp/sockets/unicorn.sock;
    server unix:///home/yinsigan/rails365_cap/shared/tmp/sockets/unicorn.sock;
}
```

3. ip_hash: 记录请求来源的ip，如果是同一个ip，下次访问的时候还是会到相同的主机，这个可以略微解决那种带cookie，session的请求的一致性问题。
```
upstream rails365 {
    ip_hash;
    server unix:///home/yinsigan/rails365/shared/tmp/sockets/unicorn.sock;
    server unix:///home/yinsigan/rails365_cap/shared/tmp/sockets/unicorn.sock;
}
```

4. hash: 上面ip_hash参数所设置的是根据相同的ip访问相同的主机，这种是根据ip地址，还有一种粒度更小的控制，可以通过任何变量来控制。

```
upstream backend {
     hash $request_uri consistent;
     server unix:///home/yinsigan/rails365/shared/tmp/sockets/unicorn.sock;
     server unix:///home/yinsigan/rails365_cap/shared/tmp/sockets/unicorn.sock;
}
```

5. down: 假如有一台主机是出了故障，或者下线了，要暂时移出，那可以把它标为down，表示请求是会略过这台主机。

```
upstream rails365 {
    server unix:///home/yinsigan/rails365/shared/tmp/sockets/unicorn.sock;
    server unix:///home/yinsigan/rails365_cap/shared/tmp/sockets/unicorn.sock down;
}
```

6. backup: 标记为备份的机器，对于备份的机器来说，其他的机器就相当于主要服务器，只要当主要服务器不可用的时候，才会用到备用服务器。
```
upstream rails365 {
    server unix:///home/yinsigan/rails365/shared/tmp/sockets/unicorn.sock;
    server unix:///home/yinsigan/rails365_cap/shared/tmp/sockets/unicorn.sock backup;
}
```

7. weight: 标记机器的权重, 默认情况下，每台主机的权重都是1，也就是说，接收请求的次数的比例是一样的。但我们可以根据主机的配置或其他情况自行调节，比如，对于配置高的主机，可以把weight值调大。
```
upstream myapp1 {
        server srv1.example.com weight=3;
        server srv2.example.com;
        server srv3.example.com;
 }
```

8.  max_fails和fail_timeout: 默认情况下，max_fails的值为1，表示的是请求失败的次数，请求1次失败就换到下台主机。另外还有一个参数是fail_timeout，表示的是请求失败的超时时间，在设定的时间内没有成功，那作为失败处理。
```
upstream rails365 {
    server unix:///home/yinsigan/rails365/shared/tmp/sockets/unicorn.sock max_fails=2;
    server unix:///home/yinsigan/rails365_cap/shared/tmp/sockets/unicorn.sock backup;
}
```

那什么情况才叫请求失败呢？

答: 可能是服务器内部错误，超时，无效的头部，或返回500以上的状态码的时候。